# Plot evolution of scores for aao vs seq assimilation (script compare_aao_vs_seq_glsd_data.py

In [ ]:
# Load datasets.
import os
import numpy as np
import xarray as xr


base_results_folder = "/storage/homefs/ct19x463/Dev/Climate/reporting/all_at_once_vs_sequential/"
years_folders = ["all_at_once_vs_sequential_62_64/", "all_at_once_vs_sequential_65_68/",
                 "all_at_once_vs_sequential_69_71/", "all_at_once_vs_sequential_72_74/",
                 "all_at_once_vs_sequential_75_77/", "all_at_once_vs_sequential_78_80/"]

prior_means = xr.open_mfdataset(paths=[os.path.join(os.path.join(base_results_folder, year_folder), "prior_means.nc") for year_folder in years_folders])
updated_means_aao = xr.open_mfdataset(paths=[os.path.join(os.path.join(base_results_folder, year_folder), "updated_means_aao.nc") for year_folder in years_folders])
updated_means_seq = xr.open_mfdataset(paths=[os.path.join(os.path.join(base_results_folder, year_folder), "updated_means_seq.nc") for year_folder in years_folders])
references = xr.open_mfdataset(paths=[os.path.join(os.path.join(base_results_folder, year_folder), "references.nc") for year_folder in years_folders])

In [ ]:
# Compute scores.
from diesel.scoring import compute_RE_score, compute_CRPS, compute_energy_score, compute_RMSE

# Loop over months in the dataset.
prior_RMSEs, aao_RMSEs, seq_RMSEs, aao_median_REs, seq_median_REs = [], [], [], [], []
for time in updated_means_aao.time:
    # Select corresponding arrays.
    prior_mean = prior_means.sel({'time': time}).temperature
    updated_mean_seq = updated_means_seq.sel({'time': time}).temperature
    updated_mean_aao = updated_means_aao.sel({'time': time}).temperature
    reference = references.sel({'time': time}).temperature

    # Stack in vector format (1D) for further computations.
    stacked_prior_mean = prior_mean.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_updated_mean_seq = updated_mean_seq.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_updated_mean_aao = updated_mean_aao.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_reference = reference.stack(stacked_dim=('latitude', 'longitude')).compute()

    # RMSE
    prior_RMSEs.append(compute_RMSE(stacked_prior_mean, stacked_reference, min_lat=-70, max_lat=70))
    aao_RMSEs.append(compute_RMSE(stacked_updated_mean_aao, stacked_reference, min_lat=-70, max_lat=70))
    seq_RMSEs.append(compute_RMSE(stacked_updated_mean_seq, stacked_reference, min_lat=-70, max_lat=70))

    RE_score_map = compute_RE_score(stacked_prior_mean, stacked_updated_mean_seq, stacked_reference, min_lat=-70, max_lat=70)
    seq_median_REs.append(np.median(RE_score_map))
    RE_score_map = compute_RE_score(stacked_prior_mean, stacked_updated_mean_aao, stacked_reference, min_lat=-70, max_lat=70)
    aao_median_REs.append(np.median(RE_score_map))

In [ ]:
# Collect into dataframe.
import pandas as pd 

scores_df = pd.DataFrame.from_dict({'time': list(updated_means_aao.time.values), 'prior RMSE': prior_RMSEs, 'aao RMSE': aao_RMSEs, 'seq RMSE': seq_RMSEs,
                          'aao median RE': aao_median_REs, 'seq median RE': seq_median_REs})

In [ ]:
# Plot results.
# RMSE.
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates


sns.set()
sns.set_style("white")
plt.rcParams["font.family"] = "serif"
plot_params = {
        'font.size': 26, 'font.style': 'oblique',
        'axes.labelsize': 'x-small',
        'axes.titlesize':'x-small',
        'legend.fontsize': 'x-small',
        'xtick.labelsize': 'x-small',
        'ytick.labelsize': 'x-small'
        }
plt.rcParams.update(plot_params)

sns.set_palette("twilight_shifted_r")


plt.figure(figsize=(35, 10))
ax = sns.lineplot(data=scores_df, x="time", y='aao RMSE', lw=4)
ax = sns.lineplot(data=scores_df, x="time", y='seq RMSE', lw=4)
ax = sns.lineplot(data=scores_df, x="time", y='prior RMSE', lw=4)

ax.tick_params(axis='x', rotation=90)
ax.set_xlim(scores_df['time'].min(), scores_df['time'].max())
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
plt.xlabel("")
plt.ylabel("RMSE")
plt.legend(["prior", "sequential", "all-at-once"])
plt.savefig("RMSE_comparison", bbox_inches="tight", pad_inches=0.1, dpi=400)
plt.show()

In [ ]:
# Plot results.
# RE score.
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates


sns.set()
sns.set_style("white")
plt.rcParams["font.family"] = "serif"
plot_params = {
        'font.size': 26, 'font.style': 'oblique',
        'axes.labelsize': 'x-small',
        'axes.titlesize':'x-small',
        'legend.fontsize': 'x-small',
        'xtick.labelsize': 'x-small',
        'ytick.labelsize': 'x-small'
        }
plt.rcParams.update(plot_params)

sns.set_palette("twilight_shifted_r")


plt.figure(figsize=(35, 10))
ax = sns.lineplot(data=scores_df, x="time", y='aao median RE', lw=4)
ax = sns.lineplot(data=scores_df, x="time", y='seq median RE', lw=4)

ax.tick_params(axis='x', rotation=90)
ax.set_xlim(scores_df['time'].min(), scores_df['time'].max())
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
plt.xlabel("")
plt.ylabel("median RE score")
plt.legend(["sequential", "all-at-once"])
plt.savefig("RE_comparison", bbox_inches="tight", pad_inches=0.1, dpi=400)
plt.show()

## Study spatial distribution.

In [ ]:
seq_RE_maps, aao_RE_maps = [], []

for time in updated_means_aao.time:
    # Select corresponding arrays.
    prior_mean = prior_means.sel({'time': time}).temperature
    updated_mean_seq = updated_means_seq.sel({'time': time}).temperature
    updated_mean_aao = updated_means_aao.sel({'time': time}).temperature
    reference = references.sel({'time': time}).temperature

    # Stack in vector format (1D) for further computations.
    stacked_prior_mean = prior_mean.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_updated_mean_seq = updated_mean_seq.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_updated_mean_aao = updated_mean_aao.stack(stacked_dim=('latitude', 'longitude')).values
    stacked_reference = reference.stack(stacked_dim=('latitude', 'longitude')).compute()

    RE_score_map = compute_RE_score(stacked_prior_mean, stacked_updated_mean_seq, stacked_reference, min_lat=-70, max_lat=70)
    print(RE_score_map)
    seq_RE_maps.append(RE_score_map)
    RE_score_map = compute_RE_score(stacked_prior_mean, stacked_updated_mean_aao, stacked_reference, min_lat=-70, max_lat=70)
    aao_RE_maps.append(RE_score_map)

In [ ]:
print(stacked_prior_mean.shape)
print(stacked_updated_mean_seq.shape)
print(stacked_reference.shape)

In [ ]:
min_lat, max_lat = -70, 70
lat_filter_inds = (references.latitude < max_lat).data & (references.latitude > min_lat).data

filtered_ref = ref = references.temperature.data [:, lat_filter_inds, :]                                         
filtered_prior_means = prior_means.temperature.data [:, lat_filter_inds, :]                                         
filtered_updated_means_aao = updated_means_aao.temperature.data [:, lat_filter_inds, :]                                         

In [ ]:
# Flatten the space dimensions.
stacked_ref = filtered_ref.reshape((filtered_ref.shape[0], filtered_ref.shape[1] * filtered_ref.shape[2]))
stacked_prior_means = filtered_prior_means.reshape((filtered_ref.shape[0], filtered_ref.shape[1] * filtered_ref.shape[2]))
stacked_updated_means_aao = filtered_updated_means_aao.reshape((filtered_ref.shape[0], filtered_ref.shape[1] * filtered_ref.shape[2]))

In [ ]:
# Cut to the shape of the reference (continents only) by getting rid of Nans.
# Shape of the NaNs is always the same, so we filter using the shape at time 0.
nan_inds = np.isnan(stacked_ref[0, :]).compute()
stacked_ref_nonan = stacked_ref.compute()[:, ~nan_inds]                                
stacked_prior_means_nonan = stacked_prior_means.compute()[:, ~nan_inds]
stacked_updated_means_aao_nonan = stacked_updated_means_aao.compute()[:, ~nan_inds]                                

In [ ]:
# Compute score.
RE_score = 1 - np.mean((stacked_updated_means_aao_nonan - stacked_ref_nonan)**2, axis=0) / np.mean((stacked_prior_means_nonan - stacked_ref_nonan)**2, axis=0)
print(RE_score.shape)